# Calculate sensitivities with the `derivnum` add-on for CO2SYS-Matlab

James Orr<br>

<img align="left" width="50%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png"><br><br>

LSCE/IPSL, CEA-CNRS-UVSQ, Gif-sur-Yvette, France

27 February 2018 <br><br>
updated: 29 June 2020 

**Abstract**: This notebook shows how to use the new `derivnum` add-on of CO2SYS-Matlab to calculate sensitivities, i.e. partial derivatives that express rates changes of computed CO2 system variables (e.g., [H+], $p$CO$_2$, $\Omega_A$) per change in the selected input variable (e.g., $A_\text{T}$, $C_\text{T}$, T, S, $K_1$, $K_2$). It uses CO2SYS-Matlab in `octave`, GNU's clone of Matlab. You can either inspect the HTML version of this file or execute its commands interactively in your browser. But for the latter, you'll need to install `jupyter notebook`, `octave`, and `oct2py`, which includes the python-octave interface called `octavemagic`. Fortunately, that installation is very easy (see below).

#### Table of Contents:
    1. Basics (install octave & oct2py and load octave)
    2. Compute sensitivities with `derivnum` add-on for CO2SYS (use ALK-DIC input pair)
    3. same but for pH-ALK
    4. same but for pCO2-DIC
    5. same but for pCO2-ALK
    6. same but for pCO2-pH

## 1. Basics

#### Run interactively

If you are visualizing this after clicking on the link to this file on github, you are seeing the HTML version of a jupyter notebook. Alternatively, you may run cells interactively and modify them if you have `jupyter notebook` installed on your machine.  To install that software, just download the anaconda open software installer for your computing platform (Windows, OS X, or Linux) from https://www.anaconda.com/ and then follow the easy install instructions at

https://docs.anaconda.com/anaconda/install/

Then just download this `jupyter notebook` file as well as the 3 routines in the src directory (`CO2SYS.m`, `errors.m`, and `derivnum.m`).  Afterwards, you'll only need to install `octave` and `oct2py` using the 1-line command in the following section.

#### Install `octavemagic`

#### To install the octavemagic funtionality, we must install `oct2py`, with the following command at the Unix prompt:

  `conda install -c conda-forge octave oct2py`
  
That command also installs `octave`. Then launch the notebook as usual with the following command:

  `jupyter notebook`

A new window or tab should then appear in your browser, showing the files in the directory from where you launched the above command. Then just click on one of the `.ipynb` files, such as this one.

Once the notebook file appears in your browser, move to any of its cells with your mouse. Run a cell by clicking on it and hitting Ctrl-Return.  Alternatively, type Shift-Return to run a cell and then move to the next one. More information on all the interactive commands is available in the Jupyter Notebook Quick Start Guide: http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html

At the top of the notebook, you'll see a number of Tabs (File, Edit, View, Insert, ...). Those tabs provide commands that will allow you to do whatever you like. Under the Help tab you'll find keyboard shortcuts for commands. Alternatively, a cheat sheet for short cuts to commands within `jupyter notebook` is available at https://zenodo.org/record/44973/files/ipynb-cheat-sheet.pdf . Or use the command palette after typing Ctrl-Shift-P.

#### Documentation for octavemagic

Details on using octavemagic are here: https://ipython.org/ipython-doc/2/config/extensions/octavemagic.html

#### Load octave magic function

Because octavemagic is now in conda's oct2py module, it is loaded with a slight modification to what is given on the above web page, i.e., now with the command below

In [1]:
%load_ext oct2py.ipython

#### List all avaiable magics

In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %octave  %octave_pull  %octave_push  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %

#### Specify the directory where you have put the Matlab routines `CO2SYS.m`, `errors.m`, and `derivnum.m`.

In [3]:
%%octave
addpath ("~/Software/MATLAB/CO2SYS-MATLAB/src")

#### Note: any notebook cell whose 1st line is `%%octave` means that all subsequent lines in that cell are in octave, not python

#### Get the documentation for using the `derivnum` routine

In [4]:
%%octave 
help derivnum

'derivnum' is a function from the file /homel/orr/Software/MATLAB/CO2SYS-MATLAB/src/derivnum.m

 derivnum()

 This subroutine computes partial derivatives of output carbonate variables

 with respect to input variables (two), plus nutrients (two), temperature and salinity,

 dissociation constants, and total boron.

...

available in the online version of the manual.  Use the command

'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW

at http://www.octave.org and via the help@octave.org

mailing list.

#### Hint:  
* double click on left of box above to hide above output
* single click to show it again 

## 2. Compute sentitivities with `derivnum` add-on for CO2SYS-Matlab ($A_\text{T}$-$C_\text{T}$ pair)

### 2.1 Specify input variables and choices

In [5]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 2300;        % ALK
    PAR2 = 2000;        % DIC
    PAR1TYPE = 1;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 2;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)
#   SI =  0;            % Total dissolved inorganic silicon (Sit)

    # Input Parameters:
    pHSCALEIN = 1;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 10; % set for K1 & K2: 10=Lueker et al. (2000); 14=Millero (2010); 15= Waters et al. (2014)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

### 2.2 Partial derivatives with respect to par1 (specified above as ALK)

In [6]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dHin/dALK dpCO2in/dALK dfCO2in/dALK dHCO3in/dALK dCO3in/dALK dCO2in/dALK dOmegaCAin/dALK dOmegaARin/dALK dxCO2in/dALK

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

-0.025752  -1.200106   -1.195933    -0.631191    0.672197    -0.041006   0.016050         0.010379       -1.224557

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [7]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));

dHout/dALK dpCO2out/dALK dfCO2out/dALK dHCO3out/dALK dCO3out/dALK dCO2out/dALK dOmegaCAout/dALK dOmegaARout/dALK dxCO2out/dALK

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

-0.025752  -1.200106   -1.195933    -0.631191    0.672197    -0.041006   0.016050         0.010379       -1.224557

### 2.2 Partial derivatives with respect to par2 (specified above as DIC)

In [8]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));

dHin/dDIC dpCO2in/dDIC dfCO2in/dDIC dHCO3in/dDIC dCO3in/dDIC dCO2in/dDIC dOmegaCAin/dDIC dOmegaARin/dDIC dxCO2in/dDIC

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

0.028288  1.470197   1.465085    1.584490    -0.634725    0.050234   -0.015155         -0.009800       1.500151

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [9]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));

dHout/dDIC dpCO2out/dDIC dfCO2out/dDIC dHCO3out/dDIC dCO3out/dDIC dCO2out/dDIC dOmegaCAout/dDIC dOmegaARout/dDIC dxCO2out/dDIC

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

0.028288  1.470197   1.465085    1.584490    -0.634725    0.050234   -0.015155         -0.009800       1.500151

### 2.3 Partial derivatives with respect to Temperature

In [10]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dHin/dT dpCO2in/dT dfCO2in/dT dHCO3in/dT dCO3in/dT dCO2in/dT dOmegaCAin/dT dOmegaARin/dT dxCO2in/dT

(nmol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

0.254392  12.669989   12.639152    -0.559386    0.424031    0.135355   0.013803         0.016707       13.325496

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [11]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dT dpCO2out/dT dfCO2out/dT dHCO3out/dT dCO3out/dT dCO2out/dT dOmegaCAout/dT dOmegaARout/dT dxCO2out/dT

(nmol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

0.254392  12.669989   12.639152    -0.559386    0.424031    0.135355   0.013803         0.016707       13.325496

### 2.4 Partial derivatives with respect to Salinity

In [12]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dHin/dS dpCO2in/dS dfCO2in/dS dHCO3in/dS dCO3in/dS dCO2in/dS dOmegaCAin/dS dOmegaARin/dS dxCO2in/dS

(nmol/kg/psu) (uatm/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

0.207253  8.173574   8.145153    0.918354    -1.140686    0.222332   -0.062840         -0.035681       8.336666

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [13]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
     printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dHin/dS dpCO2in/dS dfCO2in/dS dHCO3in/dS dCO3in/dS dCO2in/dS dOmegaCAin/dS dOmegaARin/dS dxCO2in/dS

(nmol/kg/psu) (uatm/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

0.207253  8.173574   8.145153    0.918354    -1.140686    0.222332   -0.062840         -0.035681       8.336666

### 2.1 Partial derivatives with respect to $K_1$

In [14]:
%%octave
    [b, bhead, bunits] = derivnum ('K1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dHin/dK1 dpCO2in/dK1 dfCO2in/dK1 dHCO3in/dK1 dCO3in/dK1 dCO2in/dK1 dOmegaCAin/dK1 dOmegaARin/dK1 dxCO2in/dK1

(nmol/kg/) (uatm/) (uatm/) (umol/kg/) (umol/kg/) (umol/kg/) ( /) ( /) (ppm/)

238839.155789  -234677724.151714   -233861708.528451    13377574.832229    -5359016.171885    -8018558.660305   -127952.958530         -82741.708516       -239459033.521664

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [15]:
%%octave
    [b, bhead, bunits] = derivnum ('K1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dK1 dpCO2out/dK1 dfCO2out/dK1 dHCO3out/dK1 dCO3out/dK1 dCO2out/dK1 dOmegaCAout/dK1 dOmegaARout/dK1 dxCO2out/dK1

(nmol/kg/) (uatm/) (uatm/) (umol/kg/) (umol/kg/) (umol/kg/) ( /) ( /) (ppm/)

238839.155789  -234677724.151714   -233861708.528451    13377574.832229    -5359016.171885    -8018558.660305   -127952.958530         -82741.708516       -239459033.521664

### 2.6 Partial derivatives with respect to $K_A$ (solubility product for aragonite)

In [16]:
%%octave
    [b, bhead, bunits] = derivnum ('Kspa', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dHin/dKSPA dpCO2in/dKSPA dfCO2in/dKSPA dHCO3in/dKSPA dCO3in/dKSPA dCO2in/dKSPA dOmegaCAin/dKSPA dOmegaARin/dKSPA dxCO2in/dKSPA

(nmol/kg/) (uatm/) (uatm/) (umol/kg/) (umol/kg/) (umol/kg/) ( /) ( /) (ppm/)

0.000000  0.000000   0.000000    0.000000    0.000000    0.000000   0.000000         -4805995.612422       0.000000

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [17]:
%%octave
    [b, bhead, bunits] = derivnum ('Kspa', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dKSPA dpCO2out/dKSPA dfCO2out/dKSPA dHCO3out/dKSPA dCO3out/dKSPA dCO2out/dKSPA dOmegaCAout/dKSPA dOmegaARout/dKSPA dxCO2out/dKSPA

(nmol/kg/) (uatm/) (uatm/) (umol/kg/) (umol/kg/) (umol/kg/) ( /) ( /) (ppm/)

0.000000  0.000000   0.000000    0.000000    0.000000    0.000000   0.000000         -4805995.612422       0.000000

## 3. Compute sentitivities with `derivnum` for a 2nd input pair (pH-$A_\text{T}$)

### 3.1 Specify input variables and choices

In [18]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 8.1;         % pH
    PAR2 = 2300;        % ALK
    PAR1TYPE = 3;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 1;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    #TEMPOUT = TEMPIN;
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

    # Input Parameters:
    pHSCALEIN = 1;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 15; % set for K1 & K2: 10=Lueker et al. (2000); 14=Millero (2010); 15=Waters et al. (2014)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

### 3.2 Partial derivatives with respect to H+ (since par1 is pH)

In [19]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dH dpCO2in/dH dfCO2in/dH dHCO3in/dH dCO3in/dH dCO2in/dH dOmegaCAin/dH dOmegaARin/dH dxCO2in/dH

(umol/nmol) (uatm kg/nmol) (uatm kg/nmol) (umol/nmol) (umol/nmol) (umol/nmol) (kg/nmol) (kg/nmol) (ppm kg/nmol)

30.127283  53.010721   52.826393    47.198812    -18.882819    1.811291   -0.450850         -0.291545       54.090758

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [20]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dH dpCO2out/dH dfCO2out/dH dHCO3out/dH dCO3out/dH dCO2out/dH dOmegaCAout/dH dOmegaARout/dH dxCO2out/dH

(nmol/nmol) (uatm kg/nmol) (uatm kg/nmol) (umol/nmol) (umol/nmol) (umol/nmol) (kg/nmol) (kg/nmol) (ppm kg/nmol)

0.999993  53.010384   52.826058    47.198689    -18.882685    1.811279   -0.450847         -0.291543       54.090415

### 3.3 Partial derivatives with respect to par2 (specified above as ALK)

In [21]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dALK dpCO2in/dALK dfCO2in/dALK dHCO3in/dALK dCO3in/dALK dCO2in/dALK dOmegaCAin/dALK dOmegaARin/dALK dxCO2in/dALK

(umol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

0.919834  0.158460   0.157909    0.828840    0.085580    0.005414   0.002043         0.001321       0.161689

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [22]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, PRESIN, PRESOUT,... 
                                   SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dALK dpCO2out/dALK dfCO2out/dALK dHCO3out/dALK dCO3out/dALK dCO2out/dALK dOmegaCAout/dALK dOmegaARout/dALK dxCO2out/dALK

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

0.000000  0.158461   0.157910    0.828840    0.085580    0.005414   0.002043         0.001321       0.161689

### 3.4 Partial derivatives with respect to `T` 

In [23]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dT dpCO2in/dT dfCO2in/dT dHCO3in/dT dCO3in/dT dCO2in/dT dOmegaCAin/dT dOmegaARin/dT dxCO2in/dT

(umol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

-8.261671  -0.441671   -0.424933    -13.430229    5.525867    -0.357308   0.135285         0.094565       0.006339

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [24]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dT dpCO2out/dT dfCO2out/dT dHCO3out/dT dCO3out/dT dCO2out/dT dOmegaCAout/dT dOmegaARout/dT dxCO2out/dT

(nmol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

NaN  -0.441687   -0.424948    -13.430237    5.525875    -0.357309   0.135285         0.094565       0.006323

## 4. Compute sentitivities with `derivnum` for a 3rd input pair (pCO2-$C_\text{T}$)

### 4.1 Specify input variables and choices

In [25]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 400;         % pCO2
    PAR2 = 2300;        % DIC
    PAR1TYPE = 4;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 2;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

    # Input Parameters:
    pHSCALEIN = 1;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 15; % set for K1 & K2: 10=Lueker et al. (2000); 14=Millero (2010); 15=Waters et al. (2014)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

### 4.2 Partial derivatives with respect to par1 (specified as pCO2 above)

In [26]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
  printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
  printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
  printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dpCO2 dHin/dpCO2 dfCO2in/dpCO2 dHCO3in/dpCO2 dCO3in/dpCO2 dCO2in/dpCO2 dOmegaCAin/dpCO2 dOmegaARin/dpCO2 dxCO2in/dpCO2

(umol/kg/uatm) (nmol/kg/uatm) (uatm/uatm) (umol/kg/uatm) (umol/kg/uatm) (umol/kg/uatm) ( /uatm) ( /uatm) (ppm/uatm)

-0.652024  0.018440   0.996523    0.412178    -0.446346    0.034168   -0.010657         -0.006891       1.020374

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [27]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dpCO2 dpCO2out/dpCO2 dfCO2out/dpCO2 dHCO3out/dpCO2 dCO3out/dpCO2 dCO2out/dpCO2 dOmegaCAout/dpCO2 dOmegaARout/dpCO2 dxCO2out/dpCO2

(nmol/kg/uatm) (uatm/uatm) (uatm/uatm) (umol/kg/uatm) (umol/kg/uatm) (umol/kg/uatm) ( /uatm) ( /uatm) (ppm/uatm)

0.018440  0.999994   0.996517    0.412176    -0.446344    0.034168   -0.010657         -0.006891       1.020368

### 4.3 Partial derivatives with respect to par2 (specified above as DIC)

In [28]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dDIC dHin/dDIC dfCO2in/dDIC dHCO3in/dDIC dCO3in/dDIC dCO2in/dDIC dOmegaCAin/dDIC dOmegaARin/dDIC dxCO2in/dDIC

(umol/umol) (nmol/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

1.199753  -0.003207   0.000000    0.830075    0.169925    -0.000000   0.004057         0.002624       0.000000

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [29]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dDIC dpCO2out/dDIC dfCO2out/dDIC dHCO3out/dDIC dCO3out/dDIC dCO2out/dDIC dOmegaCAout/dDIC dOmegaARout/dDIC dxCO2out/dDIC

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

-0.003207  0.000001   0.000001    0.830075    0.169925    0.000000   0.004057         0.002624       0.000001

### 4.4 Partial derivatives with respect to `T` 

In [30]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dT dHin/dT dfCO2in/dT dHCO3in/dT dCO3in/dT dCO2in/dT dOmegaCAin/dT dOmegaARin/dT dxCO2in/dT

(umol/kg/C) (nmol/kg/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

10.545137  -0.020930   0.017404    -7.202176    7.593951    -0.391775   0.185081         0.127651       0.523190

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [31]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dT dpCO2out/dT dfCO2out/dT dHCO3out/dT dCO3out/dT dCO2out/dT dOmegaCAout/dT dOmegaARout/dT dxCO2out/dT

(nmol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

-0.020930  NaN   0.017400    -7.202180    7.593955    -0.391775   0.185081         0.127651       0.523185

### 4.5 Partial derivatives with respect to `S` 

In [32]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dS dHin/dS dfCO2in/dS dHCO3in/dS dCO3in/dS dCO2in/dS dOmegaCAin/dS dOmegaARin/dS dxCO2in/dS

(umol/kg/psu) (nmol/kg/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

6.787527  0.037041   0.000000    -3.601755    3.676731    -0.074977   0.051332         0.038267       -0.004524

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [33]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dS dpCO2out/dS dfCO2out/dS dHCO3out/dS dCO3out/dS dCO2out/dS dOmegaCAout/dS dOmegaARout/dS dxCO2out/dS

(nmol/kg/psu) (uatm/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

0.037041  0.000001   0.000001    -3.601755    3.676732    -0.074977   0.051332         0.038267       -0.004523

## 5. Compute sentitivities with `derivnum` for a 4th input pair (pCO2-$A_\text{T}$)

### 5.1 Specify input variables and choices

In [34]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 400;         % pCO2
    PAR2 = 2300;        % ALK
    PAR1TYPE = 4;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 1;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

    # Input Parameters:
    pHSCALEIN = 1;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 10; % set for K1 & K2: 10=Lueker et al. (2000); 14=Millero (2010); 15=Waters et al. (2014)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

### 5.2 Partial derivatives with respect to par1 (specified as pCO2 above)

In [35]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dpCO2 dHin/dpCO2 dfCO2in/dpCO2 dHCO3in/dpCO2 dCO3in/dpCO2 dCO2in/dpCO2 dOmegaCAin/dpCO2 dOmegaARin/dpCO2 dxCO2in/dpCO2

(umol/kg/uatm) (nmol/kg/uatm) (uatm/uatm) (umol/kg/uatm) (umol/kg/uatm) (umol/kg/uatm) ( /uatm) ( /uatm) (ppm/uatm)

0.475823  0.018938   0.996523    0.737525    -0.295870    0.034168   -0.007064         -0.004568       1.020374

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [36]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dpCO2 dpCO2out/dpCO2 dfCO2out/dpCO2 dHCO3out/dpCO2 dCO3out/dpCO2 dCO2out/dpCO2 dOmegaCAout/dpCO2 dOmegaARout/dpCO2 dxCO2out/dpCO2

(nmol/kg/uatm) (uatm/uatm) (uatm/uatm) (umol/kg/uatm) (umol/kg/uatm) (umol/kg/uatm) ( /uatm) ( /uatm) (ppm/uatm)

0.018938  1.000003   0.996526    0.737526    -0.295871    0.034168   -0.007064         -0.004568       1.020377

### 5.3 Partial derivatives with respect to par2 (specified above as ALK)

In [37]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dALK dHin/dALK dfCO2in/dALK dHCO3in/dALK dCO3in/dALK dCO2in/dALK dOmegaCAin/dALK dOmegaARin/dALK dxCO2in/dALK

(umol/umol) (nmol/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

0.842070  -0.003422   0.000000    0.710478    0.131591    0.000000   0.003142         0.002032       0.000000

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [38]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dALK dpCO2out/dALK dfCO2out/dALK dHCO3out/dALK dCO3out/dALK dCO2out/dALK dOmegaCAout/dALK dOmegaARout/dALK dxCO2out/dALK

(nmol/umol) (uatm kg/umol) (uatm kg/umol) (umol/umol) (umol/umol) (umol/umol) (kg/umol) (kg/umol) (ppm kg/umol)

-0.003422  -0.000000   -0.000000    0.710478    0.131591    -0.000000   0.003142         0.002032       -0.000000

### 5.4 Partial derivatives with respect to `T` 

In [39]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dT dHin/dT dfCO2in/dT dHCO3in/dT dCO3in/dT dCO2in/dT dOmegaCAin/dT dOmegaARin/dT dxCO2in/dT

(umol/kg/C) (nmol/kg/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

-7.863218  0.003128   0.017404    -12.780944    5.309501    -0.391775   0.129840         0.090454       0.523190

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [40]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dT dpCO2out/dT dfCO2out/dT dHCO3out/dT dCO3out/dT dCO2out/dT dOmegaCAout/dT dOmegaARout/dT dxCO2out/dT

(nmol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

0.003128  NaN   0.017400    -12.780947    5.309504    -0.391775   0.129840         0.090454       0.523185

### 5.5 Partial derivatives with respect to `S` 

In [41]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTCO2/dS dHin/dS dfCO2in/dS dHCO3in/dS dCO3in/dS dCO2in/dS dOmegaCAin/dS dOmegaARin/dS dxCO2in/dS

(umol/kg/psu) (nmol/kg/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

-4.945052  0.057071   0.000000    -7.082428    2.212353    -0.074977   0.023117         0.019083       -0.004524

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [42]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dS dpCO2out/dS dfCO2out/dS dHCO3out/dS dCO3out/dS dCO2out/dS dOmegaCAout/dS dOmegaARout/dS dxCO2out/dS

(nmol/kg/psu) (uatm/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

0.057071  0.000015   0.000014    -7.082423    2.212347    -0.074976   0.023117         0.019082       -0.004509

## 6. Compute sentitivities with `derivnum` for a 4th input pair (pCO2-pH)

### 6.1 Specify input variables and choices

In [43]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 400;         % pCO2
    PAR2 = 8.0;         % pH
    PAR1TYPE = 4;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 3;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

    # Input Parameters:
    pHSCALEIN = 1;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 10; % set for K1 & K2: 10=Lueker et al. (2000); 14=Millero (2010); 15=Waters et al. (2014)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

### 6.2 Partial derivatives with respect to par1 (specified as pCO2 above)

In [44]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dpCO2 dTCO2/dpCO2 dfCO2in/dpCO2 dHCO3in/dpCO2 dCO3in/dpCO2 dCO2in/dpCO2 dOmegaCAin/dpCO2 dOmegaARin/dpCO2 dxCO2in/dpCO2

(umol/kg/uatm) (umol/kg/uatm) (uatm/uatm) (umol/kg/uatm) (umol/kg/uatm) (umol/kg/uatm) ( /uatm) ( /uatm) (ppm/uatm)

4.901099  4.585271   0.996523    4.201107    0.349996    0.034168   0.008357         0.005404       1.020374

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [45]:
%%octave
    [b, bhead, bunits] = derivnum ('par1', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dpCO2 dpCO2out/dpCO2 dfCO2out/dpCO2 dHCO3out/dpCO2 dCO3out/dpCO2 dCO2out/dpCO2 dOmegaCAout/dpCO2 dOmegaARout/dpCO2 dxCO2out/dpCO2

(nmol/kg/uatm) (uatm/uatm) (uatm/uatm) (umol/kg/uatm) (umol/kg/uatm) (umol/kg/uatm) ( /uatm) ( /uatm) (ppm/uatm)

0.000000  1.000000   0.996523    4.201107    0.349996    0.034168   0.008357         0.005404       1.020374

### 6.3 Partial derivatives with respect to par2 (specified above as pH)

In [46]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dH dTCO2/dH dfCO2in/dH dHCO3in/dH dCO3in/dH dCO2in/dH dOmegaCAin/dH dOmegaARin/dH dxCO2in/dH

(umol/nmol) (umol/nmol) (uatm kg/nmol) (umol/nmol) (umol/nmol) (umol/nmol) (kg/nmol) (kg/nmol) (ppm kg/nmol)

-230.498347  -196.043958   0.000000    -168.044289    -27.999668    0.000000   -0.668526         -0.432307       0.000000

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [47]:
%%octave
    [b, bhead, bunits] = derivnum ('par2', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN, ...
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dH dpCO2out/dH dfCO2out/dH dHCO3out/dH dCO3out/dH dCO2out/dH dOmegaCAout/dH dOmegaARout/dH dxCO2out/dH

(nmol/nmol) (uatm kg/nmol) (uatm kg/nmol) (umol/nmol) (umol/nmol) (umol/nmol) (kg/nmol) (kg/nmol) (ppm kg/nmol)

NaN  NaN   NaN    -167.935249    -28.122471    0.013762   -0.671458         -0.434203       0.410984

### 6.4 Partial derivatives with respect to `T` 

In [48]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dT dTCO2/dT dfCO2in/dT dHCO3in/dT dCO3in/dT dCO2in/dT dOmegaCAin/dT dOmegaARin/dT dxCO2in/dT

(umol/kg/C) (umol/kg/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

-0.095854  -6.910969   0.017404    -10.913501    4.394307    -0.391775   0.107403         0.074707       0.523190

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [49]:
%%octave
    [b, bhead, bunits] = derivnum ('T', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dT dpCO2out/dT dfCO2out/dT dHCO3out/dT dCO3out/dT dCO2out/dT dOmegaCAout/dT dOmegaARout/dT dxCO2out/dT

(nmol/kg/C) (uatm/C) (uatm/C) (umol/kg/C) (umol/kg/C) (umol/kg/C) ( /C) ( /C) (ppm/C)

NaN  NaN   0.017404    -10.913501    4.394307    -0.391775   0.107403         0.074707       0.523190

### 6.5 Partial derivatives with respect to `S` 

In [50]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{1:9});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{1:9});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(1:9));


dTAlk/dS dTCO2/dS dfCO2in/dS dHCO3in/dS dCO3in/dS dCO2in/dS dOmegaCAin/dS dOmegaARin/dS dxCO2in/dS

(umol/kg/psu) (umol/kg/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

14.275673  7.781113   0.000000    4.289345    3.566744    -0.074977   0.061120         0.042869       -0.004524

#### Test to see if same results for OUT conditions when TEMPOUT = TEMPIN

In [51]:
%%octave
    [b, bhead, bunits] = derivnum ('S', PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPIN,... 
                                   PRESIN, PRESOUT, SI, PO4,...
                                   pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
# Print (nicely formatted):
    printf("%s %s %s %s %s %s %s %s %s \n", bhead{10:18});
    printf("%s %s %s %s %s %s %s %s %s \n", bunits{10:18});
    printf("%f  %f   %f    %f    %f    %f   %f         %f       %f \n", b(10:18));


dHout/dS dpCO2out/dS dfCO2out/dS dHCO3out/dS dCO3out/dS dCO2out/dS dOmegaCAout/dS dOmegaARout/dS dxCO2out/dS

(nmol/kg/psu) (uatm/psu) (uatm/psu) (umol/kg/psu) (umol/kg/psu) (umol/kg/psu) ( /psu) ( /psu) (ppm/psu)

-0.000000  -0.000000   -0.000000    4.289345    3.566744    -0.074977   0.061120         0.042869       -0.004524